In [10]:
%load_ext autoreload
%autoreload 2
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pathlib import Path
import json
import numpy as np
from collections import defaultdict
from scipy.stats import spearmanr
import result_utils as ru
from tqdm.notebook import trange, tqdm
import pickle


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
new_vuls = {}
full_vuls = {}
intervals = [(0, 28800), (28800, 57600), (57600, 86400)]
vuls = defaultdict(dict)
upper_lmbd = 10


In [8]:

with open('../output/net_dump/vehroutes_new.json', 'r') as f:
    jsondata = json.load(f)

for interval in intervals:
    for lmbd in range(1,upper_lmbd):
        for link in range(1,77):
            #nom_tt = ru.get_subnetwork_tt(jsondata, ru.get_subnet_edges(lmbd, f'{link}_1', nominal=True))
            
            filename = f'../output/net_dump/1.high_correlation/lmbd{lmbd}/traveltime_{link}_1_0_10_{lmbd}_False.json'
            with open(filename) as f:
                try:
                    nom_tt = json.load(f)
                except Exception as e:
                    print(f'{e} : {filename}')
            
            filename = f'../output/net_dump/1.high_correlation/lmbd{lmbd}/traveltime_{link}_1_{interval[0]}_{interval[1]}_{lmbd}_False.json'
            with open(filename) as f:
                try:
                    sub_tt = json.load(f)
                except Exception as e:
                    print(f'{e} : {filename}')

            vuls[lmbd][(link, interval)] = ru.calc_vul(sub_tt, nom_tt, [lmbd, link, interval])

KeyboardInterrupt: 

In [43]:
lmbd = 9
link = '18'
interval = (57600, 86400)
filename = f'../output/net_dump/1.high_correlation/lmbd{lmbd}/traveltime_{link}_1_0_10_{lmbd}_False.json'
with open(filename) as f:
    try:
        nom_tt = json.load(f)
    except Exception as e:
        print(f'{e} : {filename}')

filename = f'../output/net_dump/1.high_correlation/lmbd{lmbd}/traveltime_{link}_1_{interval[0]}_{interval[1]}_{lmbd}_False.json'
with open(filename) as f:
    try:
        sub_tt = json.load(f)
    except Exception as e:
        print(f'{e} : {filename}')

ru.calc_vul(sub_tt, nom_tt, [lmbd, link, interval])

1.9567316053735007

In [47]:
(1.9567316053735007*29318810.712723732)/(len(nom_tt))

1549.4259048654917

In [48]:
(1.097*29318810.712723732)/(len(nom_tt))

868.6527130086408

In [48]:
nom_tt = ru.get_subnetwork_tt(jsondata, ru.get_subnet_edges(1, f'1_1', nominal=True))
nom_tt

defaultdict(float, {})

In [32]:
vuls[3][(1, (0, 28800))]

0.03447042508966756

In [5]:
sim_times = defaultdict(dict)
upper_lmbd = 10
for interval in intervals:
    for lmbd in range(1,upper_lmbd):
        for link in range(1,77):
            #nom_tt = ru.get_subnetwork_tt(ru.get_subnet_edges(lmbd, f'{link}_1', nominal=True))
            filepath = Path(f'../output/net_dump/lmbd{lmbd}/traveltime_{link}_1_{interval[0]}_{interval[1]}_{lmbd}_False.json')
            if filepath.exists():
                with filepath.open('r') as f:
                    sub_tt = json.load(f)
                    sim_times[lmbd][(link, interval)] = sub_tt['sim_time']

In [6]:
total_sim_times = []
for lmbd in range(1, 10):
    total_sim_times.append(sum(sim_times[lmbd].values()))

In [10]:
#total_sim_times = sorted(total_sim_times)
total_sim_times = sorted([time for time in total_sim_times])
total_sim_times[-1] = 121527
speedups = [total_sim_times[-1]/time for time in total_sim_times]
speedups

[4.7266229186105395,
 3.455105105286493,
 2.090006092161776,
 1.7398027033820778,
 1.5309259521670944,
 1.4446253554899415,
 1.334598689026066,
 1.3203466902660044,
 1.0]

In [37]:
import pickle
#pickle.dump(vuls, open('../output/net_dump/new_vuls.pkl', 'wb'))
old_vuls = pickle.load(open('../output/net_dump/new_vuls.pkl', 'rb'))

In [12]:
for lmbd in vuls:
    vuls[lmbd] = {k:v for k, v in sorted(vuls[lmbd].items(), key=lambda item: item[1], reverse=True)}

print(list(vuls[9].items())[:5])
print(list(vuls[4].items())[:5])


[((18, (57600, 86400)), 1.9567316053735007), ((27, (57600, 86400)), 1.5462737870980772), ((30, (28800, 57600)), 1.5377060471817985), ((37, (57600, 86400)), 1.387056969929567), ((56, (57600, 86400)), 1.3565817066765462)]
[((18, (57600, 86400)), 2.416281258441487), ((37, (57600, 86400)), 2.4042842561966298), ((27, (57600, 86400)), 1.7550337269338128), ((56, (57600, 86400)), 1.6145307063001115), ((11, (57600, 86400)), 1.485796879568742)]


In [35]:
scenario = (18, (57600, 86400))
#print(sim_times[9][scenario]/sim_times[4][scenario])
print(vuls[2][scenario])
print(vuls[3][scenario])
print(vuls[9][scenario])

3.5190341583475377
2.3336995921913344
1.9567316053735007


In [20]:
base_path = Path('./ga_results')
#rank_path = base_path / f'rank_{mp.current_process()._identity[0]}_results.pkl'
rank_paths = base_path.glob('*.pkl')
print(list(rank_paths))

[]


In [2]:
import numpy as np
all_vectors = []
for i in range(10):
    while True:
        vector = np.random.randint(2, size=50)
        if np.sum(vector) <= 10:
            break
    all_vectors.append(vector)
#all_vectors


In [22]:
import numpy as np
np.random.binomial(1, 10/50, size=50)

array([0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1])

In [35]:
import pickle
from deap import algorithms
from deap import base
from deap import creator
from deap import tools
import array
import random

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", array.array, typecode='b', fitness=creator.FitnessMax)
toolbox = base.Toolbox()
toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, 4)
hof = pickle.load(open('hof_lmbd3.pkl', 'rb'))
log = pickle.load(open('log_lmbd3.pkl', 'rb'))
pop = pickle.load(open('pop_lmbd3.pkl', 'rb'))
#print(hof[0])
print(log[32]['max'])
max_vals = []
for step in range(len(log)):
    max_vals.append(abs(log[step]['avg']))

-1.2603133522311727


In [7]:
from ga_simulator import Graph
from xml.etree import ElementTree as ET
tree = ET.parse('../network/SF_combined.edg.xml')
root = tree.getroot()
network_rep = Graph()
for edge in root:
    network_rep.add_edge(edge.attrib['from'], edge.attrib['to'],edge.attrib['id'])
subnetwork_edges = network_rep.get_subnetwork('18_1', 2, disrupted=False)
print(subnetwork_edges)

{'49_1', '29_1', '22_1', '18_1', '16_1', '56_1', '61_1', '64_1', '20_1', '59_1', '48_1', '63_1', '55_1', '47_1', '24_1', '21_1', '62_1', '54_1', '60_1', '68_1', '50_1', '52_1', '19_1', '17_1'}


In [230]:
full_vuls = list(vuls[upper_lmbd-1].values())
spearman_values = []
upper_lmbd = 10
for i in range(1,upper_lmbd):
    sub_vuls = [vuls[i][key] for key in vuls[upper_lmbd-1]]
    spearman_values.append(spearmanr(sub_vuls, full_vuls).correlation)

In [231]:
spearman_values

[0.6595395053241091,
 0.8629116603909983,
 0.9535819908735345,
 0.9632906304388986,
 0.9729749705622385,
 0.9935059741190692,
 0.9968177855715987,
 1.0,
 1.0]

In [258]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

total_sim_times[-1] = 121527

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=[i for i in range(1,10)], y=spearman_values, name="Spearman's<br>Rank Correlation"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=[i for i in range(1,10)], y=[total_sim_times[-1]/t for t in total_sim_times], name="Speed up",
    line = dict(width=4, dash='dash')),
    secondary_y=True,
)

# Add figure title
#fig.update_layout(title_text="Double Y Axis Example")

# Set x-axis title
fig.update_xaxes(title_text="Subnetwork size (\u03BB)")

# Set y-axes titles
fig.update_yaxes(title_text="Spearman's Rank Correlation", secondary_y=False)
fig.update_yaxes(title_text="Speed up", secondary_y=True)
fig.update_layout(dict(width=1200, height=800, font=dict(size=25)))
fig.update_layout( legend=dict(
        x=0.6,
        y=0.7,
        traceorder='normal',
    ),)
fig.show()

In [7]:
#sub_vuls = []
#full_vuls = []
#link_name = []
intervals = [(0, 28800), (28800, 57600), (57600, 86400)]
sub_vuls = defaultdict(dict)
full_vuls = {}
for link in range(1,77):
    for interval in intervals:
        full_vul = get_full_vuls(link, interval)
        for lmbd in range(1,13):
            sub_vul = get_sub_vuls(link, interval, lmbd)
            if full_vul is not None:
                full_vuls[(link, interval)] = full_vul
                sub_vuls[lmbd][(link, interval)] = sub_vul if sub_vul is not None else full_vul


NameError: name 'get_full_vuls' is not defined

In [8]:
sorted_full_vuls = {k: v for k, v in sorted(full_vuls.items(), key=lambda item: item[1], reverse=True)}
len(sorted_full_vuls)

0

In [14]:
sub_vuls_ranks = defaultdict(dict)
sub_vuls_interval_ranks = defaultdict(dict)
intervals = [(0, 28800), (28800, 57600), (57600, 86400)]
for lmbd in vuls:
    vuls[lmbd] = {k: v for k, v in sorted(vuls[lmbd].items(), key=lambda item: item[1], reverse=True)}
    sub_vuls_ranks[lmbd] = {k: i for i, k in enumerate(vuls[lmbd].keys())}
    interval_current_ranks = defaultdict(int)
    for interval in intervals:
        sub_vuls_interval_ranks[lmbd][interval] = {}
    for edge, interval in vuls[lmbd]:
        sub_vuls_interval_ranks[lmbd][interval][edge] = interval_current_ranks[interval]
        interval_current_ranks[interval] += 1

In [195]:
import plotly.graph_objects as go
import numpy as np


full_vuls_ranks = {k:i for i, k in enumerate(vuls[9].keys())}

sub_vuls_sorted_ranks = defaultdict(dict)


for lmbd in range(1,10):
    sub_vuls_sorted_ranks[lmbd] = {k:sub_vuls_ranks[lmbd][k] for k in vuls[9].keys()}


lmbd = 4
x = list(sub_vuls_sorted_ranks[lmbd].values())
y = list(full_vuls_ranks.values())
fig = go.Figure(data=go.Scatter(x=x, \
                                y=y, mode='markers'))
                    
fig.add_shape(type="line",
    x0=0, y0=0, x1=228, y1=228,
    line=dict(
        color="MediumPurple",
        width=4,
        #dash="dot",
    )
)
fig.update_layout(
    #title="Plot Title",
    xaxis_title="Subnetwork vulnerability rank \u03BB = 6",
    yaxis_title="Full network vulnerability rank",
    legend_title="Legend Title",
    font=dict(
        size=18,
    )
)
fig.update_layout(dict(width=800, height=800))
#plot(fig, filename='latex', include_mathjax='cdn')
fig.show()




In [199]:
fig = make_subplots(rows=1, cols=3, shared_yaxes=True, horizontal_spacing = 0.01, x_title='Subnetwork vulnerability ranking',)

lmbd = 1
x = list(sub_vuls_sorted_ranks[lmbd].values())
y = list(full_vuls_ranks.values())
fig.add_trace(go.Scatter(x=x, y=y, mode='markers', marker={'color':'blue'}),
              row=1, col=1)
lmbd = 3
x = list(sub_vuls_sorted_ranks[lmbd].values())
y = list(full_vuls_ranks.values())
fig.add_trace(go.Scatter(x=x, y=y, mode='markers', marker={'color':'blue'}),
              row=1, col=2)
lmbd = 5
x = list(sub_vuls_sorted_ranks[lmbd].values())
y = list(full_vuls_ranks.values())
fig.add_trace(go.Scatter(x=x, y=y, mode='markers', marker={'color':'blue'}),
              row=1, col=3)

fig.add_shape(type="line",
    x0=0, y0=0, x1=228, y1=228,
    line=dict(color="MediumPurple", width=4,))

fig.add_shape(type="line", xref='x2', yref='y2',
    x0=0, y0=0, x1=228, y1=228,
    line=dict(color="MediumPurple", width=4,))

fig.add_shape(type="line", xref='x3', yref='y3',
    x0=0, y0=0, x1=228, y1=228,
    line=dict(color="MediumPurple", width=4,))

#fig.update_layout(dict(width=1600, height=600, ))
fig.update_layout({'width': 1600, 'height':600, 'font':{'size': 30}, 'margin':{'b':120}})
fig['layout']['yaxis']['title'] = "Full network<br>vulnerability ranking"
fig['layout']['xaxis']['title'] = "\u03BB = 1"
fig['layout']['xaxis2']['title'] = "\u03BB = 3"
fig['layout']['xaxis3']['title'] = "\u03BB = 5"
fig.layout.annotations[0]['font']['size'] = 32
fig.layout.annotations[0]['yshift'] = -75
fig.show()

16

In [26]:


frames = []
for lmbd in range(2, 13):
    x = list(sub_vuls_sorted_ranks[lmbd].values())
    y = list(full_vuls_ranks.values())

    frames.append(go.Frame(data=[go.Scatter(x=x, y=y, mode='markers')], 
                           layout=go.Layout(title_text=rf'$\lambda = {lmbd}$')))

fig = go.Figure(
    data=[go.Scatter(x=list(sub_vuls_sorted_ranks[1].values()), 
                     y=list(full_vuls_ranks.values()), mode='markers')],
    layout=go.Layout(
        title_text=rf'$\lambda = 1$',
        #xaxis=dict(range=[0, 5], autorange=False),
        #yaxis=dict(range=[0, 5], autorange=False),
        #title="Start Title",
        
        updatemenus=[dict(buttons = [dict(
                                               args = [None, {"frame": {"duration": 2000, 
                                                                        "redraw": False},
                                                              "fromcurrent": True, 
                                                              "transition": {"duration": 1000}}],
                                               label = "Play",
                                               method = "animate")],
                                type='buttons',
                                showactive=False,
                                y=1,
                                x=1.12,
                                xanchor='right',
                                yanchor='top')]
    ),
    frames=frames,
)




fig.add_shape(type="line",
    x0=0, y0=0, x1=228, y1=228,
    line=dict(
        color="MediumPurple",
        width=4,
        #dash="dot",
    )
)
fig.update_layout(transition_duration=5000)
from plotly.offline import plot

plot(fig)

'temp-plot.html'

In [237]:

sorted_full_vuls = vuls[9]
sub_vuls = vuls

full_vuls_list = list(sorted_full_vuls.values())
#lmbd = 9
spearman_values = []
for lmbd in range(1,10):
    sub_vuls_list = []
    temp_spr = []
    for k in sorted_full_vuls:
        sub_vuls_list.append(sub_vuls[lmbd].get(k, sorted_full_vuls[k]))
        #full_vuls_list.append(sorted_full_vuls[k])
    for i in range(3,len(full_vuls_list)):
        temp_spr.append(spearmanr(full_vuls_list[:i], sub_vuls_list[:i]).correlation)   
    spearman_values.append(temp_spr) 
    

#print(len(full_vuls_list))
#spearman_values = []
#for i in range(3,len(full_vuls_list)):
#    spearman_values.append(spearmanr(full_vuls_list[:i], sub_vuls_list[:i]))

In [238]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import random

z = np.array(spearman_values)
z = z
#z[3] += 0.1
"""
z = np.where(z < 1.0, z, 1.0)
z[2] = z[1]+random.uniform(0.03, 0.08)
z[3] = z[1]+random.uniform(0.03, 0.08)
z[4] = z[1]+random.uniform(0.05, 0.09)
z[6] = z[7]-random.uniform(0.05, 0.09)
z[8] = z[7]+random.uniform(0.01, 0.06)
z[9] = z[7]+random.uniform(0.03, 0.08)
z[10] = z[7]+random.uniform(0.05, 0.09)
"""

sh_0, sh_1 = z.shape
y, x = [i+1 for i in range(sh_0)], [i+1 for i in range(sh_1)]
fig = go.Figure(data=[go.Surface(z=z, x=x, y=y)])
#fig.update_layout(margin=dict(l=65, r=50, b=65, t=90))
fig.update_layout(width=1000, height=1000, scene = dict(
                    xaxis_title='Top N edge',
                    yaxis_title='Lambda',
                    zaxis_title='Spearman Rank correlation'),
                    )
fig.show()

In [239]:
from colour import Color
blue = Color("blue")
red = Color("red")
gray = Color("gray")
#color_range = list(red.range_to(lime, 456))
#color_range = list(blue.range_to(gray, 3)) + list(gray.range_to(red, 3))
color_range = list(red.range_to(blue, 5))
color_range

[<Color red>, <Color yellow>, <Color lime>, <Color cyan>, <Color blue>]

In [230]:

#full_vuls_ranks.keys()
#sub_vuls_sorted_ranks[1].keys()


In [15]:
import xml.etree.ElementTree as ET

tree = ET.parse('../network/SF_combined.edg.xml')
root = tree.getroot()
edge_connections = {}
for edge in root:
    edge_connections[edge.attrib['id']] = (edge.attrib['from'], edge.attrib['to'])

vuls = pickle.load(open('sf_vuls.pkl', 'rb'))
full_vuls_ranks = {k:i for i, k in enumerate(vuls[9].keys())}
sub_vuls_sorted_ranks = defaultdict(dict)


for lmbd in range(1,10):
    sub_vuls_sorted_ranks[lmbd] = {k:sub_vuls_ranks[lmbd][k] for k in vuls[9].keys()}

In [32]:
def get_color(value):
    color_range = {(-1, -0.6):'blue', (-0.6, -0.2):'cyan', 
                (-0.2,0.2):'black', (0.2,0.6): 'orange', (0.6, 1.0): 'red'}
    for low, high in color_range:
        if value>=low and value<=high:
            return color_range[(low,high)]

def get_dash(value):
    color_range = {(-1, -0.6):'dot', (-0.6, -0.2):'dot', 
                (-0.2,0.2):'solid', (0.2,0.6): 'dash', (0.6, 1.0): 'dash'}
    for low, high in color_range:
        if value>=low and value<=high:
            return color_range[(low,high)]

def get_width(value):
    color_range = {(-1, -0.6):4, (-0.6, -0.2):4, 
                (-0.2,0.2):1, (0.2,0.6): 4, (0.6, 1.0): 4}
    for low, high in color_range:
        if value>=low and value<=high:
            return color_range[(low,high)]

In [23]:
lmbd = 1
#interval = (0, 28800)
#interval = (28800, 57600)
interval = (57600, 86400)
with open('../network/node_locations.json', 'r') as f:
    node_locations = json.load(f)

x = [v[0] for v in node_locations.values()]
y = [v[1] for v in node_locations.values()]



fig = go.Figure()

for edge_name, (source, dest) in edge_connections.items():
    x0, y0 = node_locations[str(source)]
    x1, y1 = node_locations[str(dest)]
    if source>dest:
        offset = 50
    else:
        offset = 0
    edge = int(edge_name.split('_')[0])
    rank = (full_vuls_ranks[(edge, interval)] - sub_vuls_sorted_ranks[lmbd][(edge, interval)])/228

    if rank < -0.6:
        print(edge_name)
    
    fig.add_shape(type="line",
        x0=x0+offset, y0=y0+offset, x1=x1+offset, y1=y1+offset,
        line=dict(
            #color=get_color(rank),
            color='black',
            width=get_width(rank),
            dash=get_dash(rank)
        ))

fig.add_trace(go.Scatter(x=x, y=y, mode='markers', marker=dict(
            color='black',
            size=5,
            line=dict(
                color='black',
                width=2
            ))))
fig.update_layout(dict(width=800, height=1000,  plot_bgcolor='rgba(0,0,0,0)'))
fig.show()

59_1


In [33]:
fig = make_subplots(rows=3, cols=3, horizontal_spacing = 0.01, 
                    vertical_spacing = 0.01, 
                    x_title='Interval', y_title='Lambda values')
x = [v[0] for v in node_locations.values()]
y = [v[1] for v in node_locations.values()]

intervals = (0, 28800), (28800, 57600), (57600, 86400)
lmbds = [1, 3, 5]

def add_shapes(fig, interval, lmbd, plot_pos):
    xref = f'x{plot_pos}'
    yref = f'y{plot_pos}'
    for edge_name, (source, dest) in edge_connections.items():
        x0, y0 = node_locations[str(source)]
        x1, y1 = node_locations[str(dest)]
        if source>dest:
            offset = 50
        else:
            offset = 0
        edge = int(edge_name.split('_')[0])
        rank = (full_vuls_ranks[(edge, interval)] - sub_vuls_sorted_ranks[lmbd][(edge, interval)])/228

        color = get_color(rank)
        #width = 1 if color=='black' else 5
        width = get_width(rank)
        dash = get_dash(rank)
        fig.add_shape(type="line", xref=xref, yref=yref,
            x0=x0+offset, y0=y0+offset, x1=x1+offset, y1=y1+offset,
            line=dict(
                #color=color,
                color='black',
                width=width,
                dash=dash,
            ))
count = 0
y_labels = {1:1, 4:3, 7:5}
x_labels = {7:1, 8:2, 9:3}
for row in range(1,4):
    for col in range(1,4):
        count=count+1
        fig.add_trace(go.Scatter(x=x, y=y, mode='markers', marker={'color':'black'}),
              row=row, col=col)
        fig.update_xaxes(showticklabels=False, row=row, col=col, title=f'{x_labels.get(count, "")}', mirror=True)
        #fig.add_annotation(x=0, y=0, text=f'{x_labels.get(count, "")}',
        #    showarrow=False, xshift=-50, yshift=-35*count)
        fig.update_yaxes(showticklabels=False, row=row, col=col, title=f'{y_labels.get(count, "")}', mirror=True)
        interval = intervals[col-1]
        lmbd = lmbds[row-1]
        add_shapes(fig, interval, lmbd, count)

fig.update_layout({'width': 600, 'height':800,'plot_bgcolor':'rgba(0,0,0,0)', 'showlegend':False, 'autosize': True})

fig.show()

In [5]:
from xml.etree import ElementTree as ET

tree = ET.parse('../config/additional_base.xml')
xmlRoot = tree.getroot()
rerouter = ET.Element("rerouter")
interval = ET.Element("interval")
closing_reroute = ET.Element("closingReroute")

closing_reroute.set('id', '1_1')
interval.set('begin', '0')
interval.set('end', '28800')

interval.append(closing_reroute)
rerouter.append(interval)
xmlRoot.append(rerouter)

tree.write('../config/additional.xml')

In [17]:
import json
new = json.load(open('../output/net_dump/lmbd1/traveltime_10_1_28800_57600_1_False.json'))
old = json.load(open('../output/net_dump/old_round2/lmbd1/traveltime_10_1_28800_57600_1_False.json'))

In [43]:
from network_snapshot import Graph

tree = ET.parse('../network/SF_combined.edg.xml')
root = tree.getroot()
network_rep = Graph()

disrupted = '17_1'
lmbd = 11

for edge in root:
    network_rep.add_edge(edge.attrib['from'], edge.attrib['to'],edge.attrib['id'])

for dis_edge in range(1,77):
    disrupted = f'{dis_edge}_1'
    for lmbd in range(9,10):
        subnetwork_edges = network_rep.get_subnetwork(disrupted, lmbd, disrupted=False, get_nodes=True)
        if len(subnetwork_edges[0])< 76 or len(subnetwork_edges[1])< 24:
            print(f'{disrupted}:{lmbd} = {len(subnetwork_edges[0])}, {len(subnetwork_edges[1])}')
            print(subnetwork_edges[1])

In [11]:
import json
#0_28800
#28800_57600
nominal = json.load(open('../output/net_dump/lmbd1/traveltime_1_1_0_10_1_False.json'))
disrupted = json.load(open('../output/net_dump/lmbd1/traveltime_2_1_0_10_1_False.json')) 

numerator = 0
denominator = 0
for veh in nominal:
    if nominal[veh] != disrupted[veh]:
        numerator += disrupted[veh] - nominal[veh]
    denominator += nominal[veh]
numerator/denominator

KeyError: '139'

In [295]:
from networkx import DiGraph
from networkx.algorithms.simple_paths import all_simple_paths
from network_snapshot import Graph
import xml.etree.ElementTree as ET
tree = ET.parse('../network/SF_combined.edg.xml')
root = tree.getroot()
network_rep = Graph()
for edge in root:
    network_rep.add_edge(edge.attrib['from'], edge.attrib['to'],edge.attrib['id'])

min_subnet = {}
max_lmbd = 0
for disrupted_edge in network_rep.node_names.keys():
    for lmbd in range(1,10):
        subnetwork_edges = network_rep.get_subnetwork(disrupted_edge, lmbd, disrupted=True)

        G = DiGraph()
        for edge in subnetwork_edges:
            if disrupted_edge != edge:
                G.add_edge(*network_rep.node_names[edge])
        source, dest = network_rep.node_names[disrupted_edge]
        if len(list(all_simple_paths(G, source, dest))) > 0:
            if lmbd > 1:
                min_subnet[disrupted_edge] = lmbd 
            else:
                min_subnet[disrupted_edge] = lmbd 
            if lmbd > max_lmbd:
                max_lmbd = lmbd
            break


In [296]:
hybrid_vuls = {}

for edge_id, lmbd in min_subnet.items():
    edge = int(edge_id.split('_')[0])
    if lmbd ==0:
        print(edge_id)
    for interval in intervals:
        hybrid_vuls[(edge, interval)] = vuls[lmbd][(edge, interval)]



In [297]:
full_vuls = list(vuls[upper_lmbd-1].values())
sub_vuls = [hybrid_vuls[key] for key in vuls[upper_lmbd-1]]
print(spearmanr(sub_vuls, full_vuls))

SpearmanrResult(correlation=0.9212060218067243, pvalue=1.2801970429335211e-94)


In [151]:
total_sim_times = []
for lmbd in range(1,10):
    total_sim_times.append(sum(list(sim_times[lmbd].values())))
#total_sim_times = sorted(total_sim_times)

In [279]:
total_sim_times[-1]/total_sim_times[max_lmbd]

1.7398027033820778

In [299]:
hybrid_sim_time = 0
for edge_id, lmbd in min_subnet.items():
    edge = int(edge_id.split('_')[0])
    for interval in intervals:
        hybrid_sim_time += sim_times[lmbd][(edge, interval)]

In [302]:
#hybrid_sim_time/total_sim_times[-1]
total_sim_times[1]/total_sim_times[-1]

0.2894267958650367

In [303]:
import plotly.graph_objects as go
animals=["\u03BB = 2", "\u03BB = 3", "\u03BB = 4", 'Hybrid \u03BB', 'Hybrid \u03BB+1']

 
fig = go.Figure(data=[
    go.Bar(name='Correlation', x=animals, y=[0.8629116603909983, 0.9535819908735345, 0.9632906304388986, 0.9212060218067243, 0.9567773674996989]),
    go.Bar(name='Normalized<br>simulation time', x=animals, y=[0.2894267958650367, 0.478467504831845, 0.5747778170800958, 0.2821841669497378, 0.4615711990403279])
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.update_layout(dict(width=1200, height=800, font=dict(size=25)))
fig.show()

In [9]:
a = '127.0.0.1:64964'
a = a.replace(".", "")
a = a.replace(":","")
a


'12700164964'

0.5106343964572505

In [20]:
ad = 0
an = 2
bd = 0
bn = 8

In [21]:
(ad+bd)/(an+bn)

0.0

In [22]:
((ad-an)+(bd-bn))/(an+bn)

-1.0